<div class="alert alert-block alert-success">

<!-- -info-warning-success-danger>   --> <font size=5>
<b>
   Проверка пола по имени - 16-12-23
        </b>    </div>
<hr>

In [174]:
# pd.set_option('display.expand_frame_repr', True)
# pd.set_option('display.max_rows', 50)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)
# pd.set_option('display.max_colwidth', None)

In [175]:
# Даты как в ЭКСЕЛЬ
# df_cloud_dead_data_pusto['Дата рождения_облако'] = pd.to_datetime(df_cloud_dead_data_pusto['Дата рождения_облако']).dt.strftime('%d.%m.%Y')

In [176]:
import pandas as pd
import numpy as np
import warnings
from IPython.display import display_markdown
warnings.filterwarnings('ignore')
from rusgenderdetection import get_gender

####
<div class="alert alert-block alert-success"> <font size=5>
<b>подключаем облако и 605й (1c)
    </b>    </div>
<hr>

In [177]:
file_cloud = 'Облако_и_1С.csv'
df_cloud = pd.read_csv(file_cloud, delimiter=';', encoding='cp1251')

print(df_cloud.shape)


(16181, 81)


In [178]:
# df_cloud.columns.tolist()

In [179]:
df_cloud = df_cloud.loc[~df_cloud['ФИО пациента_облако'].isna()][['ID пациента_облако',
 'Дата создания заявки_облако',
 'Дата включения пациента в Реестр_облако',
 'ФИО пациента_облако',
 'Пол_облако',
 'Дата рождения_облако',
 'Гражданский статус_облако',]]
print(df_cloud.shape)

(15181, 7)


####
<div class="alert alert-block alert-success"> <font size=3>
пробъем по имени
    </b>    </div>
<hr>

In [180]:
df_cloud['name'] = df_cloud['ФИО пациента_облако'].map(lambda x: x.split(' ')[1])
df_cloud['last'] = df_cloud['ФИО пациента_облако'].map(lambda x: x[-3:])
df_cloud['Пол_облако'] = df_cloud['Пол_облако'].str.lower()
df_cloud['correct_gender'] = df_cloud['name'].map(lambda x: get_gender(x))
df_cloud.head(2)

,ID пациента_облако,Дата создания заявки_облако,Дата включения пациента в Реестр_облако,ФИО пациента_облако,Пол_облако,Дата рождения_облако,Гражданский статус_облако,name,last,correct_gender
0,30000007827048,NaN,23.11.2020,Скрыпник Виктор Александрович,мужской,1959-11-28,лицо без определенного места жительства,Виктор,вич,1
1,20862846,NaN,07.10.2021,Попова Любовь Терентьевна,женский,1932-08-26,Москвич,Любовь,вна,0


In [183]:
df_cloud['correct_gender'] = np.where(df_cloud['correct_gender'] == 1, 'мужской', (np.where(df_cloud['correct_gender'] == 0, 'женский', 'н/у')))
df_cloud.head(2)

,ID пациента_облако,Дата создания заявки_облако,Дата включения пациента в Реестр_облако,ФИО пациента_облако,Пол_облако,Дата рождения_облако,Гражданский статус_облако,name,last,correct_gender
0,30000007827048,NaN,23.11.2020,Скрыпник Виктор Александрович,мужской,1959-11-28,лицо без определенного места жительства,Виктор,вич,мужской
1,20862846,NaN,07.10.2021,Попова Любовь Терентьевна,женский,1932-08-26,Москвич,Любовь,вна,женский


In [184]:
df_identif_name = df_cloud.loc[df_cloud['correct_gender'] != 'н/у']
df_identif_name.shape

(14060, 10)

####
<div class="alert alert-block alert-success"> <font size=3>
ищем НЕСОВПАДЕНИЕ по имени
    </b>    </div>
<hr>

In [185]:
df_error_gender_by_name = df_identif_name.loc[df_identif_name['correct_gender'] != df_identif_name['Пол_облако']]
df_error_gender_by_name.shape

(47, 10)

####
<div class="alert alert-block alert-success"> <font size=3>
ищем НЕСОВПАДЕНИЕ по ОКОНЧАНИЮ отчества
    </b>    </div>
<hr>

In [186]:
df_dont_identif_name = df_cloud.loc[df_cloud['correct_gender'] == 'н/у']
df_dont_identif_name.shape

(1121, 10)

In [187]:
# -----------несовпаденеие вна и женского пола

In [188]:
df_error_gender_by_vna = df_dont_identif_name.loc[
                                              (df_dont_identif_name['last'] == 'вна')
                                            & (df_dont_identif_name['Пол_облако'] == 'мужской')   
        ]
df_error_gender_by_vna.shape

(27, 10)

In [189]:
# -----------несовпаденеие вич и мужского пола

In [190]:
df_error_gender_by_vich = df_dont_identif_name.loc[
                                          (df_dont_identif_name['last'] == 'вич')
                                        & (df_dont_identif_name['Пол_облако'] == 'женский')   
        ]
df_error_gender_by_vich.shape

(0, 10)

####
<div class="alert alert-block alert-success"> <font size=3>
соберем все ДФ с ошибками в единый дф
    </b>    </div>
<hr>

In [191]:
frames = [df_error_gender_by_name, df_error_gender_by_vna, df_error_gender_by_vich]

In [192]:
df_error_gender = pd.concat(frames)
df_error_gender.shape

(74, 10)

In [193]:
df_error_gender['Дата рождения_облако'] = pd.to_datetime(df_error_gender['Дата рождения_облако']).dt.strftime('%d.%m.%Y')
df_error_gender.head()

,ID пациента_облако,Дата создания заявки_облако,Дата включения пациента в Реестр_облако,ФИО пациента_облако,Пол_облако,Дата рождения_облако,Гражданский статус_облако,name,last,correct_gender
679,2243395102,NaN,21.04.2023,Симонова Зинаида Николаевна,мужской,01.12.1941,Москвич,Зинаида,вна,женский
915,10273975,NaN,29.12.2020,Подушкина Нина Никитична,мужской,11.01.1940,Москвич,Нина,чна,женский
962,2244632315,NaN,27.02.2022,Борисов Борис Денисович,женский,30.07.1936,Москвич,Борис,вич,мужской
1375,16388594,NaN,06.08.2023,Толчинский Михаил Олегович,женский,05.08.1953,Москвич,Михаил,вич,мужской
1398,24661174,NaN,31.10.2023,Абаничев Евгений Викторович,женский,05.10.1960,Москвич,Евгений,вич,мужской


In [173]:
df_error_gender.to_excel('df_error_gender.xlsx')